#**LAB 11 : Hidden Markov Model**

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Please refer to the following [article](http://www.adeveloperdiary.com/data-science/machine-learning/introduction-to-hidden-markov-model/) to understand Hidden Markov Model

Here we will be dealing with 3 major problems : 
  
  1. Evaluation Problem
  2. Learning Problem
  3. Decoding Problem

1. Evaluation Problem : Implementation of Forward and Backward Algorithm

## Forward

![Forward](forward.jpg)

## Backward

![Backward](backward.jpg)

In [5]:
data = pd.read_csv('data_python.csv') ## Read the data, change the path accordingly

V = data['Visible'].values

# Transition Probabilities
a = np.array(((0.54, 0.46), (0.49, 0.51)))

# Emission Probabilities
b = np.array(((0.16, 0.26, 0.58), (0.25, 0.28, 0.47)))

# Equal Probabilities for the initial distribution
initial_distribution = np.array((0.5, 0.5)) ## Write your code here


def forward(V, a, b, initial_distribution):
    alpha = np.zeros((V.shape[0], a.shape[0]))
    ## Write your code here
    
    alpha[0, :] = initial_distribution * b[:, V[0]]
    for t in range(1, V.shape[0]):
        alpha[t, :] = (alpha[t-1] @ a) * b[:, V[t]]
    
    return alpha


alpha = forward(V, a, b, initial_distribution)
print("alpha:")
print(alpha)


def backward(V, a, b):
    beta = np.zeros((V.shape[0], a.shape[0]))

    ## Write your code here
    beta[V.shape[0]-1] = np.ones(a.shape[0])
    for t in range(V.shape[0]-2, -1, -1):
        for j in range(a.shape[0]):
            beta[t, j] = np.sum(a[j, :] * beta[t+1] * b[:, V[t+1]])
    return beta


beta = backward(V, a, b)
print("beta")
print(beta)


alpha:
[[8.00000000e-002 1.25000000e-001]
 [2.71570000e-002 2.81540000e-002]
 [1.65069392e-002 1.26198572e-002]
 [8.75653677e-003 6.59378003e-003]
 [4.61649960e-003 3.47369232e-003]
 [2.43311103e-003 1.83073126e-003]
 [1.28234420e-003 9.64864889e-004]
 [6.75844805e-004 5.08520930e-004]
 [3.56196241e-004 2.68010114e-004]
 [1.87729137e-004 1.41251652e-004]
 [9.89404851e-005 7.44450603e-005]
 [5.21454461e-005 3.92354139e-005]
 [2.74826583e-005 2.06785741e-005]
 [1.44844194e-005 1.08984050e-005]
 [7.63384683e-006 5.74387913e-006]
 [4.02333128e-006 3.02724551e-006]
 [9.50546790e-007 9.50495728e-007]
 [5.67842140e-007 4.33342042e-007]
 [3.01003967e-007 2.26639558e-007]
 [1.58685405e-007 1.19402560e-007]
 [8.36344763e-008 6.29285781e-008]
 [1.97593813e-008 1.97583215e-008]
 [5.29142730e-009 5.36649662e-009]
 [1.42660806e-009 1.44787155e-009]
 [2.36772066e-010 3.48663550e-010]
 [1.73247192e-010 1.34764774e-010]
 [4.14929379e-011 4.15586480e-011]
 [2.48065559e-011 1.89323811e-011]
 [3.62758511e

2. Learning Problem : Implementation of Baum Welch Algorithm

## Baum-Welch

![Baum-Welch](bw.jpg)

In [6]:
def baum_welch(V, a, b, initial_distribution, n_iter=100):

  M = a.shape[0]
  T = len(V)
  K = b.shape[1]
    
  ## Write your code here
  for n in range(n_iter):
    alpha = forward(V, a, b, initial_distribution)
    beta = backward(V, a, b)
    Xi = np.zeros((M, M, T - 1))
    for t in range(T-1):
      den = np.dot(np.dot(alpha[t, :].T, a) * b[:, V[t + 1]].T, beta[t + 1, :])
      for i in range(M):
        num = alpha[t, i] * a[i, :] * b[:, V[t+1]] * beta[t+1, :].T
        Xi[i, :, t] = num
      Xi[:, :, t] /= den

    gamma = np.sum(Xi, axis=1)
    a = np.sum(Xi, axis=2)/np.sum(gamma, axis=1).reshape((-1, 1))
    
    gamma = np.hstack((gamma, np.sum(Xi[:, :, T - 2], axis=0).reshape((-1, 1))))
    
    den = np.sum(gamma, axis=1).reshape(-1, 1)
    for k in range(K):
      b[:, k] = np.sum(gamma[:, V==k], axis=1)
    
    b = np.divide(b, den)
    
  return (a,b)

data = pd.read_csv('data_python.csv')

V = data['Visible'].values

# Transition Probabilities
a = np.ones((2, 2))
a = a / np.sum(a, axis=1)

# Emission Probabilities
b = np.array(((1, 3, 5), (2, 4, 6)))
b = b / np.sum(b, axis=1).reshape((-1, 1))

# Equal Probabilities for the initial distribution
initial_distribution = np.array((0.5, 0.5))

a, b = baum_welch(V, a, b, initial_distribution, n_iter=100)
print({"a": a, "b": b})


{'a': array([[0.53816345, 0.46183655],
       [0.48664443, 0.51335557]]), 'b': array([[0.16277513, 0.26258073, 0.57464414],
       [0.2514996 , 0.27780971, 0.47069069]])}


3. Decoding Problem : Implementation of Viterbi Algorithm

## Viterbi Algorithm

![Viterbi](vit.jpg)

In [7]:
def viterbi(V, a, b, initial_distribution):

  ## Write your code here
  T = V.shape[0]
  M = a.shape[0]

  omega = np.zeros((T, M))
  omega[0, :] = np.log(initial_distribution * b[:, V[0]])

  prev = np.zeros((T - 1, M))
  for t in range(1, T):
      for j in range(M):
          probability = omega[t - 1] + np.log(a[:, j]) + np.log(b[j, V[t]])
          prev[t - 1, j] = np.argmax(probability)
          omega[t, j] = np.max(probability)

  S = np.zeros(T)
  last_state = np.argmax(omega[T - 1, :])
  S[0] = last_state
  backtrack_index = 1
  for i in range(T - 2, -1, -1):
      S[backtrack_index] = prev[i, int(last_state)]
      last_state = prev[i, int(last_state)]
      backtrack_index += 1

  S = np.flip(S, axis=0)

  result = []
  for s in S: result.append('B' if s else 'A')

  return result


data = pd.read_csv('data_python.csv')

V = data['Visible'].values

# Transition Probabilities
a = np.ones((2, 2))
a = a / np.sum(a, axis=1)

# Emission Probabilities
b = np.array(((1, 3, 5), (2, 4, 6)))
b = b / np.sum(b, axis=1).reshape((-1, 1))

# Equal Probabilities for the initial distribution
initial_distribution = np.array((0.5, 0.5))

a, b = baum_welch(V, a, b, initial_distribution, n_iter=100)

result = viterbi(V, a, b, initial_distribution)
print(result)


['B', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'A', 'A', 'A', 'B', 'A', 'B', 'B', 'A', 'A', 'A', 'B', 'A', 'B', 'A', 'A', 'B', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'A', 'A', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'A', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'A', 'A', 'B', 'A', 'B', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',

4. Use the built-in **hmmlearn** package to fit the data and generate the result using the decoder

In [8]:
!pip install hmmlearn==0.2.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 129 kB 5.1 MB/s 


In [9]:
## Write your code heredata = pd.read_csv('data_python.csv')
from hmmlearn.hmm import MultinomialHMM

V = data['Visible'].values

# Transition Probabilities
a = np.ones((2, 2))
a = a / np.sum(a, axis=1)

# Emission Probabilities
b = np.array(((1, 3, 5), (2, 4, 6)))
b = b / np.sum(b, axis=1).reshape((-1, 1))

# Equal Probabilities for the initial distribution
initial_distribution = np.array((0.5, 0.5))

In [10]:
model = MultinomialHMM(n_components=2)
model.startprob_ = initial_distribution
model.transmat_ = a
model.emissionprob_ = b

In [11]:
_, seq = model.decode(np.array([V]).reshape(-1, 1))

result = []
for s in seq: result.append('B' if s else 'A')
print(result)

['B', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'A', 'B', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'A', 'A', 'A', 'A', 'B', 'A', 'A', 'A', 'B', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'A', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'A', 'B', 'A', 'B', 'B', 'A', 'B', 'A', 'A', 'A', 'B', 'A', 'B', 'A', 'B', 'B', 'B', 'A', 'A', 'B', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'A', 'B', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'A', 'A', 'B', 'A', 'B', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'B', 'A', 'B', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'B', 'A',